In [6]:
import tensorflow as tf
import os

import csv
import numpy as np
import pandas as pd
import pickle

import datetime
import time

import random

#### 1. CNN construction

In [12]:
class CNN(object):
    def __init__(self, padding_size=500, embedding_size=100, file_path='matrix_Folder/', 
                 num_classes=5, filter_sizes = [2,3,4], num_filters = 128, dropout_keep_prob = 0.5):
        
        self.input_x = tf.placeholder(tf.float32, [None, padding_size, embedding_size, 1], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.test = tf.placeholder(tf.float32, [None, padding_size, embedding_size, 1], name="testing")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        l2_loss = tf.constant(0.0)

        pooled_outputs = []
        
        # for each filter size
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                # W is the filter matrix
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.input_x,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # provide non-linearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Max pooling
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, padding_size - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        print(pooled_outputs[0])

        # concatenate all the outputs into a long features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs,3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        
        # drop out
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # output 
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            
            #do wx + b matrix multiplication
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
        
        # compute loss and accuracy
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses)

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

#### 2.Parameters Configuration

In [20]:
# parameters

batch_size = 64
num_epochs = 10
dropout_keep_prob = 0.5
split_percentage = 0.01

num_checkpoints = 100
eval_interval  = 100
checkpoint_interval= 100

Xfile = "matrix_Folder/"
Yfile = "songs.csv"

#### 3.helper functions
Those functions are helper functions which are used in loading data and padding vectors to make sure their size be the same.

In [21]:
# those are helper functions that are used in the next cell

# 1.load x and load y
def load_X(folder_path,nums_file=25000):
    files = []
    for i in range(nums_file):
        filename = folder_path + "matrix" + str(i) + ".txt"
        files.append(filename)
            
    return files

def load_Y(filename):
    df = pd.read_csv(filename)
    classes = list(df['genre'])
    genres = sorted(list(set(classes)))
    print(genres)
    
    res = []
    for i, label in enumerate(classes):
        index = genres.index(label)
        label = [0] * len(genres)
        label[index] = 1
        res.append(label)
        
    return res

# 2.padding to make each matrix has the same size(500)
def padding0(file):
    with open(file, 'rb') as f:
        lyrics = pickle.load(f)
        
        if len(lyrics) <= 500:
            padding_num = 500 - len(lyrics)
            for j in range(padding_num):
                zero_row = np.zeros(100).tolist()
                lyrics.append(zero_row)
            return np.expand_dims(np.array(lyrics), axis=2)
        else:
            lyrics = lyrics[0:500]
            return np.expand_dims(np.array(lyrics), axis=2)

In [22]:
x = load_X(Xfile)  # x is a list of filenames
y = load_Y(Yfile)  #shape: 25000 x 5

#shuffle data and split data into train/test parts

temp = list(zip(x, y))
random.shuffle(temp)
x_shuffled, y_shuffled = zip(*temp)

split_index = -1 * int(split_percentage * float(len(y)))

x_train, x_test = x_shuffled[:split_index], x_shuffled[split_index:]
y_train, y_test = y_shuffled[:split_index], y_shuffled[split_index:]


# padding the vector
x_test = [padding0(file) for file in x_test]

print("Train/Test: {:d}/{:d}".format(len(y_train), len(y_test)))

['Country', 'Jazz', 'Metal', 'Pop', 'Rock']
Train/Test: 24750/250


#### 4. Training

In [25]:
# functions: 
#    for a single training step/ testing step
#    and evaluate the loss and accuracy on a batch of data and updating the model parameters.

def train_step(x_batch, y_batch):
    input_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: dropout_keep_prob
    }
    
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy], input_dict)

    cur_time = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(cur_time, step, loss, accuracy))
    res = str(cur_time) + ", " + str(step) + ", " + str(loss) + ", " + str(accuracy) + '\n'
    
    file2.write(res)
    train_summary_writer.add_summary(summaries, step)

    
def test_step(x_batch, y_batch, writer=None):
    input_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: 1.0
    }
    
    step, summaries, loss, accuracy = sess.run(
        [global_step, test_summary_op, cnn.loss, cnn.accuracy], input_dict)

    cur_time = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(cur_time, step, loss, accuracy))
    res = str(cur_time) + ", " + str(step) + ", " + str(loss) + ", " + str(accuracy) + '\n'
    file1.write(res)
    
    if writer:
        writer.add_summary(summaries, step)


#  function:
#  generate the batch
def batch_iterator(data, batch_size, num_epochs, shuffle=True):
   
    data_size = len(data)
    num_batches_per_epoch = int(data_size-1)/batch_size) + 1
    
    for epoch in range(num_epochs):
        if shuffle:
            shuffled_data  = data
            random.shuffle(shuffled_data)
        else:
            shuffled_data = data

        for batch_num in range(num_batches_per_epoch):
            start = batch_num * batch_size
            end = min((batch_num + 1) * batch_size, data_size)
            
            x_iter  = [padding0(pair[0]) for pair in shuffled_data[start:end]]
            y_iter  = [pair[1] for pair in shuffled_data[start:end]]
            
            yield list(zip(x_iter, y_iter))  

In [26]:
testFile = "test.txt"
trainFile = "train.txt"

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)

    with sess.as_default():
        cnn = CNN()

        global_step = tf.Variable(0, name="global_step", trainable=False)
        
        # optimizer
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # record the gradient values
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # the output directory to save 1)summaries and 2)models/checkpoints
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
        
        # 1)save the summary
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        test_summary_op = tf.summary.merge([loss_summary, acc_summary])
        test_summary_dir = os.path.join(out_dir, "summaries", "test")
        test_summary_writer = tf.summary.FileWriter(test_summary_dir, sess.graph)

        # 2)save the checkpoint
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        
        # Initialize the variables
        sess.run(tf.global_variables_initializer())
        
        file1 = open(testFile, 'w') 
        file2 = open(trainFile, 'w')

        # Generate batches
        batches = batch_iterator(list(zip(x_train, y_train)), batch_size, num_epochs)
        
        # for each batch, train
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            
            if current_step % eval_interval == 0:
                print("\nEvaluation:")
                test_step(x_test, y_test, writer=test_summary_writer)
                print("")
                
            if current_step % checkpoint_interval == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

        file1.close()
        file2.close()

print("finish")

Tensor("conv-maxpool-2/pool:0", shape=(?, 1, 1, 128), dtype=float32)
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/spars

2018-12-07T15:53:05.877766: step 100, loss 2.04039, acc 0.36

Saved model checkpoint to C:\Users\hooke\Desktop\ecen765_Project\runs\1544219414\checkpoints\model-100

2018-12-07T15:53:08.649570: step 101, loss 4.13138, acc 0.375
2018-12-07T15:53:10.305376: step 102, loss 5.61799, acc 0.359375
2018-12-07T15:53:11.818057: step 103, loss 5.14301, acc 0.234375
2018-12-07T15:53:13.689765: step 104, loss 4.76927, acc 0.296875
2018-12-07T15:53:15.267473: step 105, loss 5.49963, acc 0.203125
2018-12-07T15:53:17.818065: step 106, loss 4.46147, acc 0.390625
2018-12-07T15:53:19.674755: step 107, loss 5.41682, acc 0.234375
2018-12-07T15:53:21.372448: step 108, loss 4.96335, acc 0.265625
2018-12-07T15:53:22.894179: step 109, loss 4.03028, acc 0.359375
2018-12-07T15:53:24.655535: step 110, loss 5.52626, acc 0.21875
2018-12-07T15:53:26.219290: step 111, loss 4.36683, acc 0.328125
2018-12-07T15:53:27.868826: step 112, loss 5.33858, acc 0.234375
2018-12-07T15:53:29.677690: step 113, loss 4.44745, acc 0.

2018-12-07T15:56:50.192240: step 224, loss 3.58898, acc 0.25
2018-12-07T15:56:51.872007: step 225, loss 3.12353, acc 0.296875
2018-12-07T15:56:54.033841: step 226, loss 2.93204, acc 0.390625
2018-12-07T15:56:56.737724: step 227, loss 3.20421, acc 0.359375
2018-12-07T15:56:58.749629: step 228, loss 3.90719, acc 0.21875
2018-12-07T15:57:00.861211: step 229, loss 3.43613, acc 0.359375
2018-12-07T15:57:02.896284: step 230, loss 3.59089, acc 0.34375
2018-12-07T15:57:05.063214: step 231, loss 3.46645, acc 0.265625
2018-12-07T15:57:07.589879: step 232, loss 3.13217, acc 0.328125
2018-12-07T15:57:09.301558: step 233, loss 2.79403, acc 0.359375
2018-12-07T15:57:11.780341: step 234, loss 3.19096, acc 0.28125
2018-12-07T15:57:14.128132: step 235, loss 3.61232, acc 0.328125
2018-12-07T15:57:15.822272: step 236, loss 3.21901, acc 0.34375
2018-12-07T15:57:17.707017: step 237, loss 2.96953, acc 0.40625
2018-12-07T15:57:19.950211: step 238, loss 2.69022, acc 0.40625
2018-12-07T15:57:22.142354: step 23

2018-12-07T16:00:40.924849: step 349, loss 2.46542, acc 0.3125
2018-12-07T16:00:42.844108: step 350, loss 1.99108, acc 0.390625
2018-12-07T16:00:44.533297: step 351, loss 2.18711, acc 0.4375
2018-12-07T16:00:46.479538: step 352, loss 1.97345, acc 0.359375
2018-12-07T16:00:48.260548: step 353, loss 2.42396, acc 0.375
2018-12-07T16:00:50.146562: step 354, loss 1.9724, acc 0.421875
2018-12-07T16:00:51.817771: step 355, loss 2.12736, acc 0.328125
2018-12-07T16:00:53.513977: step 356, loss 2.33985, acc 0.34375
2018-12-07T16:00:55.262827: step 357, loss 2.43565, acc 0.3125
2018-12-07T16:00:56.689519: step 358, loss 2.06174, acc 0.421875
2018-12-07T16:00:58.823877: step 359, loss 2.58235, acc 0.328125
2018-12-07T16:01:00.913892: step 360, loss 2.41188, acc 0.40625
2018-12-07T16:01:02.746990: step 361, loss 2.42759, acc 0.328125
2018-12-07T16:01:04.344005: step 362, loss 2.43725, acc 0.34375
2018-12-07T16:01:05.918337: step 363, loss 2.51727, acc 0.296875
2018-12-07T16:01:07.479111: step 364, 

2018-12-07T16:03:31.643410: step 475, loss 1.73884, acc 0.390625
2018-12-07T16:03:32.834671: step 476, loss 1.63987, acc 0.375
2018-12-07T16:03:34.004214: step 477, loss 1.31807, acc 0.53125
2018-12-07T16:03:35.075686: step 478, loss 1.35933, acc 0.53125
2018-12-07T16:03:36.262215: step 479, loss 1.62795, acc 0.375
2018-12-07T16:03:37.489474: step 480, loss 1.77043, acc 0.40625
2018-12-07T16:03:38.609885: step 481, loss 1.572, acc 0.421875
2018-12-07T16:03:39.645889: step 482, loss 1.53112, acc 0.46875
2018-12-07T16:03:40.719069: step 483, loss 1.67897, acc 0.421875
2018-12-07T16:03:41.808405: step 484, loss 2.12042, acc 0.25
2018-12-07T16:03:42.908972: step 485, loss 1.75772, acc 0.40625
2018-12-07T16:03:44.123589: step 486, loss 1.54729, acc 0.40625
2018-12-07T16:03:45.160772: step 487, loss 1.6025, acc 0.484375
2018-12-07T16:03:46.337719: step 488, loss 1.68253, acc 0.4375
2018-12-07T16:03:47.670161: step 489, loss 1.29219, acc 0.4375
2018-12-07T16:03:48.835469: step 490, loss 1.546

...

2018-12-07T17:06:36.824739: step 3293, loss 0.757389, acc 0.71875
2018-12-07T17:06:38.043255: step 3294, loss 0.818154, acc 0.625
2018-12-07T17:06:39.303781: step 3295, loss 0.560879, acc 0.78125
2018-12-07T17:06:40.393149: step 3296, loss 0.779433, acc 0.6875
2018-12-07T17:06:41.467295: step 3297, loss 0.781352, acc 0.734375
2018-12-07T17:06:42.551261: step 3298, loss 0.789344, acc 0.734375
2018-12-07T17:06:43.943380: step 3299, loss 0.684742, acc 0.703125
2018-12-07T17:06:45.075374: step 3300, loss 0.614667, acc 0.828125

Evaluation:
2018-12-07T17:06:46.316057: step 3300, loss 0.610425, acc 0.76

Saved model checkpoint to C:\Users\hooke\Desktop\ecen765_Project\runs\1544219414\checkpoints\model-3300

2018-12-07T17:06:48.526288: step 3301, loss 0.810205, acc 0.703125
2018-12-07T17:06:50.063161: step 3302, loss 0.464908, acc 0.875
2018-12-07T17:06:51.365857: step 3303, loss 0.656904, acc 0.796875
2018-12-07T17:06:52.560096: step 3304, loss 0.696129, acc 0.6875
2018-12-07T17:06:53.746343

2018-12-07T17:09:28.715209: step 3412, loss 0.684649, acc 0.734375
2018-12-07T17:09:30.227719: step 3413, loss 0.670214, acc 0.734375
2018-12-07T17:09:31.932243: step 3414, loss 0.784455, acc 0.734375
2018-12-07T17:09:33.459326: step 3415, loss 0.532873, acc 0.796875
2018-12-07T17:09:35.113994: step 3416, loss 0.871306, acc 0.609375
2018-12-07T17:09:36.757175: step 3417, loss 0.608016, acc 0.796875
2018-12-07T17:09:38.257468: step 3418, loss 0.644827, acc 0.765625
2018-12-07T17:09:39.756408: step 3419, loss 0.73189, acc 0.671875
2018-12-07T17:09:41.370967: step 3420, loss 0.673624, acc 0.765625
2018-12-07T17:09:43.202159: step 3421, loss 0.7128, acc 0.6875
2018-12-07T17:09:44.922763: step 3422, loss 0.843609, acc 0.640625
2018-12-07T17:09:46.394299: step 3423, loss 0.782843, acc 0.703125
2018-12-07T17:09:48.213989: step 3424, loss 0.504753, acc 0.828125
2018-12-07T17:09:50.025686: step 3425, loss 0.802909, acc 0.734375
2018-12-07T17:09:51.418167: step 3426, loss 0.746989, acc 0.75
2018

2018-12-07T17:12:29.394717: step 3534, loss 0.589946, acc 0.734375
2018-12-07T17:12:30.394193: step 3535, loss 0.729462, acc 0.75
2018-12-07T17:12:31.621454: step 3536, loss 0.570139, acc 0.78125
2018-12-07T17:12:32.652310: step 3537, loss 0.563468, acc 0.8125
2018-12-07T17:12:33.738175: step 3538, loss 0.545506, acc 0.8125
2018-12-07T17:12:35.083281: step 3539, loss 0.690312, acc 0.671875
2018-12-07T17:12:36.091369: step 3540, loss 0.757328, acc 0.75
2018-12-07T17:12:37.168361: step 3541, loss 0.703853, acc 0.75
2018-12-07T17:12:38.276282: step 3542, loss 0.672489, acc 0.75
2018-12-07T17:12:39.296347: step 3543, loss 0.52234, acc 0.84375
2018-12-07T17:12:40.328817: step 3544, loss 0.699579, acc 0.703125
2018-12-07T17:12:41.387029: step 3545, loss 0.667189, acc 0.78125
2018-12-07T17:12:42.413353: step 3546, loss 0.399984, acc 0.90625
2018-12-07T17:12:43.492927: step 3547, loss 0.566697, acc 0.78125
2018-12-07T17:12:44.543319: step 3548, loss 0.603638, acc 0.71875
2018-12-07T17:12:45.67

2018-12-07T17:14:54.901754: step 3657, loss 0.764317, acc 0.671875
2018-12-07T17:14:56.216649: step 3658, loss 0.64463, acc 0.734375
2018-12-07T17:14:57.408918: step 3659, loss 0.563164, acc 0.796875
2018-12-07T17:14:58.774940: step 3660, loss 0.669868, acc 0.6875
2018-12-07T17:14:59.854959: step 3661, loss 0.66523, acc 0.765625
2018-12-07T17:15:01.250174: step 3662, loss 0.624489, acc 0.765625
2018-12-07T17:15:02.611822: step 3663, loss 0.639616, acc 0.703125
2018-12-07T17:15:03.665747: step 3664, loss 0.831593, acc 0.65625
2018-12-07T17:15:04.841332: step 3665, loss 0.550293, acc 0.78125
2018-12-07T17:15:06.057795: step 3666, loss 0.593595, acc 0.796875
2018-12-07T17:15:07.234568: step 3667, loss 0.607727, acc 0.734375
2018-12-07T17:15:08.349471: step 3668, loss 0.584476, acc 0.78125
2018-12-07T17:15:09.881041: step 3669, loss 0.648814, acc 0.71875
2018-12-07T17:15:10.964515: step 3670, loss 0.591075, acc 0.78125
2018-12-07T17:15:12.185616: step 3671, loss 0.577834, acc 0.796875
2018

2018-12-07T17:17:36.089933: step 3779, loss 0.575449, acc 0.78125
2018-12-07T17:17:37.917072: step 3780, loss 0.713827, acc 0.734375
2018-12-07T17:17:39.397058: step 3781, loss 0.695843, acc 0.75
2018-12-07T17:17:40.935274: step 3782, loss 0.663774, acc 0.75
2018-12-07T17:17:42.328031: step 3783, loss 0.616439, acc 0.71875
2018-12-07T17:17:43.462773: step 3784, loss 0.661896, acc 0.75
2018-12-07T17:17:44.716545: step 3785, loss 0.668068, acc 0.71875
2018-12-07T17:17:46.061256: step 3786, loss 0.68344, acc 0.734375
2018-12-07T17:17:47.577468: step 3787, loss 0.768148, acc 0.71875
2018-12-07T17:17:48.977200: step 3788, loss 0.812513, acc 0.6875
2018-12-07T17:17:50.355928: step 3789, loss 0.677032, acc 0.734375
2018-12-07T17:17:52.057364: step 3790, loss 0.530892, acc 0.84375
2018-12-07T17:17:53.492901: step 3791, loss 0.63235, acc 0.703125
2018-12-07T17:17:54.674801: step 3792, loss 0.653669, acc 0.71875
2018-12-07T17:17:56.169338: step 3793, loss 0.673092, acc 0.734375
2018-12-07T17:17: